# Matrix Factorization Recommedation System  

We designed this recommendation system as the user based system. Based on the users previous eatting habbit to generate the resteraunts satisfied their taste. The matrix factorization is taking the product of the customer matrix and the Restaurant matrix as the basic evaluation matrix, based on which reteraunts they went to beofore to suggest the top 5 resteraunts they may goes to afterwards by searching a specific reteraunts name. 

In here we built the models by ourselves but conducted the loss function by ourselves. Also, we tried other kernal functions which have been created in the package Surprised like KNN and SVD to see how it performace.

In [2]:
# import packages
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
# Using Surprise as the matraix Factorization
import surprise  
from surprise import Reader
from surprise import Dataset
# suppress warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
business = pd.read_csv("yelp_business.csv")
business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [4]:
review = pd.read_csv("yelp_review.csv")
review.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [5]:
user_product=review.iloc[:,1:4]

In [6]:
merged_left = pd.merge(left=user_product, right=business, how='left', left_on='business_id', right_on='business_id')
merged_left.head()

,user_id,business_id,stars_x,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,categories
0,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,"""Wilensky's""",Plateau-Mont-Royal,"""34 Avenue Fairmount Ouest""",Montréal,QC,H2T 2M1,45.523333,-73.594859,4.0,84,1,Diners;Food;Restaurants;Delis
1,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,"""Tuck Shop""",Sud-Ouest,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
2,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,"""Lester's Deli""",Outremont,"""1057 Avenue Bernard""",Outremont,QC,H2V 1V1,45.522144,-73.607076,4.0,70,1,Specialty Food;Food;Sandwiches;Restaurants;Bur...
3,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,"""Five Guys""",Ville-Marie,"""698 Rue Sainte-Catherine O""",Montréal,QC,H3B 1B9,45.502510,-73.570119,3.5,61,1,Fast Food;Restaurants;Burgers
4,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,"""Maison Christian Faure""",Ville-Marie,"""355 Place Royale""",Montréal,QC,H2Y 1Z6,45.503544,-73.554469,4.5,397,1,Bakeries;Restaurants;Food;Sandwiches;Cafes;Cof...


In [7]:
AZ=merged_left.loc[merged_left['state'] == 'AZ']

## Get the datasets for Matrix factorization

In [8]:
Matrix=AZ.iloc[:,:3]
Matrix.head()

,user_id,business_id,stars_x
86,nsOf58RZjMTn8V94EQYJog,djyIZW8gVNWby8wauISkqg,5
93,ssuXFjkH4neiBgwv-oN4IA,JlNeaOymdVbE6_bubqjohg,1
94,ssuXFjkH4neiBgwv-oN4IA,0Rni7ocMC_Lg2UH0lDeKMQ,1
95,ssuXFjkH4neiBgwv-oN4IA,3uBrRcIhbhed1xftLBvabw,1
355,MzEnYCyZlRYQRISNMXTWIg,S-oLPRdhlyL5HAknBKTUcQ,4


In [9]:
Matrix.shape

(1627792, 3)

In [10]:
Matrix.rename(columns = {'stars_x':'raw_ratings'}, inplace = True) 

In [26]:
df = pd.DataFrame(Matrix)

In [27]:
## Convert the dataframe into instance because the matrix factorization need a specific data input
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(df[['user_id', 'business_id', 'raw_ratings']], reader)

In [28]:
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import os

In [29]:
class MatrixFactorization(surprise.AlgoBase):
   
    
    def __init__(self, learning_rate, n_epochs, n_factors, lmd):
        
        self.lr = learning_rate  # Gradientdescent learning rate
        self.n_epochs = n_epochs  # Gradientdescent itteration times
        self.n_factors = n_factors  # the factorized matrix(rank) [k factors in here]
        self.lmd = lmd # In order to avoid overfitting, we use lambda in here 
        
    def fit(self, trainset):
        '''after the gradient descent, we get all the value u_i and p_j '''
        
        print('Fitting data with SGD...')
        
        # randomly set the user-based and item_baed matrix.
        u = np.random.normal(0, .1, (trainset.n_users, self.n_factors))
        p = np.random.normal(0, .1, (trainset.n_items, self.n_factors))
        
        # Gradient descent
        for _ in range(self.n_epochs):
            for i, j, r_ij in trainset.all_ratings():
                err = r_ij - np.dot(u[i], p[j])
                # Adjust the u_i and p_j
                u[i] -= -self.lr * err * p[j] + self.lr * self.lmd * u[i]
                p[j] -= -self.lr * err * u[i] + self.lr * self.lmd * p[j]
                
        
        self.u, self.p = u, p
        self.trainset = trainset

    def estimate(self, i, j):
        '''predict how user i rate item j .'''
        
        # If user_i and item J are known, they we return the dot product by u_i and p_j
        # If it hasn't appeared before then we use the average rating(Cold Start problem)
        if self.trainset.knows_user(i) and self.trainset.knows_item(j):
            return np.dot(self.u[i], self.p[j])
        else:
            return self.trainset.global_mean

In [30]:
# Split the datasets
trainset, testset = train_test_split(data, test_size=.25)
# Setting the matrix that we defined above.
algo = MatrixFactorization(learning_rate=.005, n_epochs=60, n_factors=2, lmd = 0.2)

# # Train
algo.fit(trainset)

# # Prediction
predictions = algo.test(testset)

# Caculate the average of mean squared error
accuracy.mae(predictions)

Fitting data with SGD...
MAE:  1.1769


1.1768793510683229

## Try to compute the performance by using KNN

In [23]:
# Compute the MAE by using the kernal function KNN in Surprise 
algo = surprise.KNNBasic()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
MAE:  1.2194


1.2194452594849503

## Try to compute the performance by using SVD

In [24]:
# Compute the MAE by using the kernal function SVD in Surprise 
algo = surprise.SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.mae(predictions)

MAE:  1.1332


1.1332224533854074

# Conclusion:

We can see that the SVD performs the best in this situation. We want to see how does this prediction works in this case

In [66]:
## Using dataframe to take a look at it, for each User_id = customer, we have some resteraunts that we think it may suit for their customer taste
prediction=pd.DataFrame(predictions)
prediction1=prediction.drop(columns=['details'])
pred=pd.merge(left=prediction1,right=business,how='left',left_on='iid',right_on='business_id')
pred.head()

,uid,iid,r_ui,est,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,I5pKhvba6BZhpHoNqNPkIw,QuybD_bJcAB2CHcpTfREYg,5.0,3.776778,QuybD_bJcAB2CHcpTfREYg,"""Detroit Coney Grill""",NaN,"""930 W Broadway Rd""",Tempe,AZ,85282,33.407892,-111.953919,4.5,546,1,American (Traditional);Burgers;American (New);...
1,H3KluKXf1mp3etCToCJhqg,Cz3FK54yRZSiGRgiAMK9OQ,3.0,3.864981,Cz3FK54yRZSiGRgiAMK9OQ,"""The Home Depot""",NaN,"""2530 E Germann Rd""",Chandler,AZ,85286,33.281840,-111.796929,3.5,40,1,Nurseries & Gardening;Shopping;Appliances;Hard...
2,vh9an___kX9fmf6RvhlTpQ,IrXlXhmSHlIllxZSAARjLA,4.0,2.611887,IrXlXhmSHlIllxZSAARjLA,"""Flaming Kabob""",NaN,"""921 E University Dr""",Tempe,AZ,85281,33.421881,-111.925474,3.5,107,1,Mediterranean;Halal;Middle Eastern;Restaurants
3,u0Ck_gZ2bJnYF5IeTeYfHA,DUC4IaoxAVfnQrCTTh7fSw,5.0,3.142977,DUC4IaoxAVfnQrCTTh7fSw,"""East Valley Pediatric Dentistry""",NaN,"""1355 S Higley Rd, Ste 117""",Gilbert,AZ,85296,33.326776,-111.720532,4.5,37,1,Pediatric Dentists;Health & Medical;Orthodonti...
4,egcYHY7nXSog504CdOI8-w,3nudXkhgKYY92Wc8rQxxSg,1.0,3.766532,3nudXkhgKYY92Wc8rQxxSg,"""21 Cakes""",NaN,"""7001 N Scottsdale Rd, Ste 175""",Scottsdale,AZ,85253,33.539242,-111.924382,3.5,128,1,Coffee & Tea;Cupcakes;Food;Bakeries;Desserts


#### So we a specific customer goes to our website, based on his habbits, the system will provide a resteraunts' list to him. We can see that he loves Mexican foods, and the recommendation mostly are mexican foods by looking at the category. Both' Rokerij' and 'Tee Pee' are mexican foods.

In [65]:
df1=pred.sort_values(['est'],ascending=False).groupby('uid').head(3)
df1[df1['uid']=='mDMIxpocw2JFfJFa5dWH1A']

,uid,iid,r_ui,est,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
28,mDMIxpocw2JFfJFa5dWH1A,5A86y0jNE6vpKk983iWKUQ,5.0,3.968268,5A86y0jNE6vpKk983iWKUQ,"""Select Auto Body""",NaN,"""545 W Highland Ave""",Phoenix,AZ,85013,33.505507,-112.081995,5.0,91,1,Automotive;Auto Repair;Body Shops
333968,mDMIxpocw2JFfJFa5dWH1A,_WtxQbDK7B-ExGdeG-2j6Q,5.0,3.092052,_WtxQbDK7B-ExGdeG-2j6Q,"""Rokerij""",NaN,"""6335 N 16th St""",Phoenix,AZ,85016,33.530382,-112.047075,4.0,604,1,Steakhouses;Mexican;Seafood;New Mexican Cuisin...
129590,mDMIxpocw2JFfJFa5dWH1A,YtUaJkgKc1pucwdSq_DQeQ,2.0,2.679008,YtUaJkgKc1pucwdSq_DQeQ,"""Tee Pee Mexican Food""",NaN,"""4144 E Indian School Rd""",Phoenix,AZ,85018,33.495241,-111.991589,3.5,360,1,Mexican;Tex-Mex;Restaurants


### Recommendation Sysyem - Content filtering recommedation System (on-going) Omit

In [1]:
import pandas as pd
All=pd.read_csv('Pheonix_Final_v2.csv')

In [2]:
All.columns

Index([u'Unnamed: 0', u'business_id', u'Shortlisted?', u'ProjectCategory',
       u'name', u'neighborhood', u'address', u'city', u'state', u'postal_code',
       u'latitude', u'longitude', u'stars', u'review_count', u'is_open',
       u'categories', u'label', u'PARKING', u'BikeParking',
       u'HairSpecializesIn_coloring', u'CONSUMER_SERVICE', u'NoiseLevel',
       u'Alcohol', u'Ambience_romantic', u'HappyHour', u'RECOMMENDATION',
       u'DogsAllowed', u'SPECIAL_FOOD', u'AgesAllowed', u'Unnamed: 0.1',
       u'weekday', u'weekend', u'averghour', u'if_Chain', u'polarity'],
      dtype='object')

In [3]:
# Alll=pd.merge(left=All,right=text,how='inner',left_on='business_id',right_on='business_id')

In [4]:
All1=All.drop(columns=['Unnamed: 0.1','Unnamed: 0','address', 'city', 'state', 'postal_code', 'latitude','longitude','categories','business_id'])

In [5]:
All1.head()

,Shortlisted?,ProjectCategory,name,neighborhood,stars,review_count,is_open,label,PARKING,BikeParking,...,HappyHour,RECOMMENDATION,DogsAllowed,SPECIAL_FOOD,AgesAllowed,weekday,weekend,averghour,if_Chain,polarity
0,Yes,Burger,"""McDonald's""",NaN,1.0,10,1,-1,1,0,...,0,0,1,0.0,0,0.0,0.0,0.000000,1,0.152415
1,Yes,Burger,"""Charr An American Burger Bar""",NaN,3.0,232,1,-1,1,0,...,0,0,0,0.0,0,56.0,22.0,11.142857,1,0.233437
2,Yes,Burger,"""McDonald's""",NaN,3.0,34,1,-1,1,0,...,0,0,1,0.0,0,90.0,36.0,18.000000,1,0.104785
3,Yes,Other,"""Red Lobster""",NaN,2.5,37,1,0,0,1,...,0,0,0,0.0,0,56.0,23.0,11.285714,0,0.198911
4,Yes,Sandwiches,"""D'Lish Cafe""",NaN,4.0,37,0,0,0,0,...,0,0,0,0.0,0,25.0,10.0,5.000000,0,0.201747


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [7]:
listings_missing_df = All1.isnull().mean()*100

In [8]:
All1['Shortlisted?']=le.fit_transform(All1['Shortlisted?']) 
All1['ProjectCategory']=le.fit_transform(All1['ProjectCategory']) 
All1=All1.drop(columns=['neighborhood'])
All1['SPECIAL_FOOD'] = All1['SPECIAL_FOOD'].fillna(All1['SPECIAL_FOOD'].mode()[0])

In [9]:
All1.head()

,Shortlisted?,ProjectCategory,name,stars,review_count,is_open,label,PARKING,BikeParking,HairSpecializesIn_coloring,...,HappyHour,RECOMMENDATION,DogsAllowed,SPECIAL_FOOD,AgesAllowed,weekday,weekend,averghour,if_Chain,polarity
0,0,3,"""McDonald's""",1.0,10,1,-1,1,0,0,...,0,0,1,0.0,0,0.0,0.0,0.000000,1,0.152415
1,0,3,"""Charr An American Burger Bar""",3.0,232,1,-1,1,0,0,...,0,0,0,0.0,0,56.0,22.0,11.142857,1,0.233437
2,0,3,"""McDonald's""",3.0,34,1,-1,1,0,0,...,0,0,1,0.0,0,90.0,36.0,18.000000,1,0.104785
3,0,18,"""Red Lobster""",2.5,37,1,0,0,1,0,...,0,0,0,0.0,0,56.0,23.0,11.285714,0,0.198911
4,0,20,"""D'Lish Cafe""",4.0,37,0,0,0,0,0,...,0,0,0,0.0,0,25.0,10.0,5.000000,0,0.201747


In [10]:
All1.columns

Index([u'Shortlisted?', u'ProjectCategory', u'name', u'stars', u'review_count',
       u'is_open', u'label', u'PARKING', u'BikeParking',
       u'HairSpecializesIn_coloring', u'CONSUMER_SERVICE', u'NoiseLevel',
       u'Alcohol', u'Ambience_romantic', u'HappyHour', u'RECOMMENDATION',
       u'DogsAllowed', u'SPECIAL_FOOD', u'AgesAllowed', u'weekday', u'weekend',
       u'averghour', u'if_Chain', u'polarity'],
      dtype='object')

In [11]:
## Calculate the average rating
avg=All1['stars'].mean()
avg

3.4777906976744184

In [12]:
##calculate the number of votes by 65 quantile
m = All1['review_count'].quantile(0.65)
m

50.0

In [13]:
q_rest = All1.copy().loc[All1['review_count'] >= m]
q_rest.shape

(1519, 24)

In [14]:
def weighted_rating(x, m=m, avg=avg):
    v = x['review_count']
    R = x['stars']
    return (v/(v+m) * R) + (m/(m+v) * avg)

In [15]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_rest['score'] = q_rest.apply(weighted_rating, axis=1)

In [16]:
#Sort movies based on score calculated above
q_rest = q_rest.sort_values('score', ascending=False)

#Print the top 15 movies
q_rest[['name', 'review_count', 'stars', 'score']].head(15)

,name,review_count,stars,score
3765,"""Little Miss BBQ""",1463,5.0,4.949696
4286,"""Frost Gelato""",188,5.0,4.680208
558,"""A T Oasis Coffee & Tea Shop""",167,5.0,4.649261
4269,"""Tres Leches Cafe""",165,5.0,4.645998
2482,"""Bobby Q""",1940,4.5,4.474316
2879,"""Lux Central""",1770,4.5,4.471917
3300,"""Oliver's Sophisticated Bean""",94,5.0,4.471455
4207,"""Cibo""",1698,4.5,4.470761
263,"""Novel Ice Cream""",92,5.0,4.464011
1752,"""Postino Arcadia""",1186,4.5,4.458648


In [17]:
trainset=q_rest[:1064]
testset=q_rest[1064:]

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
trainset['name'] = trainset['name'].fillna('')
tfidf_matrix = tfidf.fit_transform(trainset['name'])
tfidf_matrix.shape

/Users/cindy/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(1064, 1268)

In [19]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [20]:
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [22]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(trainset.index, index=trainset['name']).drop_duplicates()

In [23]:
indices

name
"Little Miss BBQ"                        3765
"Frost Gelato"                           4286
"A T Oasis Coffee & Tea Shop"             558
"Tres Leches Cafe"                       4269
"Bobby Q"                                2482
"Lux Central"                            2879
"Oliver's Sophisticated Bean"            3300
"Cibo"                                   4207
"Novel Ice Cream"                         263
"Postino Arcadia"                        1752
"Kiss Pollos Estilo Sinaloa"              940
"Original Breakfast House"                813
"Paradise Valley Burger Company"           53
"Postino Central"                         824
"Harumi Sushi"                           4085
"Federal Pizza"                          3960
"Welcome Diner"                          3489
"Dick's Hideaway"                        1733
"La Grande Orange Pizzeria"               207
"Churn"                                  3528
"Steak 44"                               1914
"Green New American Vegetaria

In [26]:
# Function that takes in movie title as input and outputs most similar 
def get_recommendations(name, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[name]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    rest_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return q_rest.iloc[rest_indices]